In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hands model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

# Initialize MediaPipe Drawing utility
mp_drawing = mp.solutions.drawing_utils

def calculate_angle(a, b, c):
    a = np.array(a) 
    b = np.array(b) 
    c = np.array(c) 
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

# Start capturing video from the webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    
    # Flip the frame vertically for better visualization
    frame = cv2.flip(frame, 1)
    
    # Convert the BGR image to RGB and process it with MediaPipe Hands
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    # Draw hand landmarks on the image
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Calculate angles between fingers
            thumb_angle = calculate_angle([hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP].x,
                                           hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP].y],
                                          [hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].x,
                                           hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].y],
                                          [hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x,
                                           hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y])
            
            index_angle = calculate_angle([hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].x,
                                           hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y],
                                          [hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].x,
                                           hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y],
                                          [hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x,
                                           hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y])
            
            middle_angle = calculate_angle([hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].x,
                                            hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y],
                                           [hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].x,
                                            hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y],
                                           [hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x,
                                            hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y])
            
            ring_angle = calculate_angle([hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].x,
                                          hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y],
                                         [hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].x,
                                          hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].y],
                                         [hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].x,
                                          hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y])
            
            pinky_angle = calculate_angle([hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].x,
                                           hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].y],
                                          [hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].x,
                                           hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].y],
                                          [hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].x,
                                           hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y])
            
            # Count fingers by checking if they are open or closed based on the calculated angles
            fingers_open = [thumb_angle > 30, index_angle < 160, middle_angle < 160, ring_angle < 160, pinky_angle < 160]
            
            # Display count of open fingers on the image
            cv2.putText(frame, f'Fingers open: {sum(fingers_open)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Display the image
    cv2.imshow('MediaPipe Hands', frame)
    
    # Exit loop if 'q' is pressed
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# Release the webcam and close OpenCV window
cap.release()
cv2.destroyAllWindows()
